In [ ]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import string
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import time
import requests
import threading
import nltk
import json
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
from sklearn.preprocessing import normalize

In [ ]:
# downloading stop words and punctuation
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Polzovatel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Polzovatel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# getting stop words and initializing a stemmer
stop_words = stopwords.words('russian')
stemmer = SnowballStemmer(language='russian')

In [ ]:
headersForVacancy = {'Authorization': '#'}
def getVacancy(spec_words_lst, all_filtered_tokens, lock2, vacId):
  # text cleaning and getting vacancy information
  
  filtered_tokens = []
  try:
      response = requests.get(f'https://api.hh.ru/vacancies/{vacId}', headers=headersForVacancy)
  except:
      response = requests.get(f'https://api.hh.ru/vacancies/{vacId}', headers=headersForVacancy)
  if response.status_code != 200:
    print('error')
  vac = response.json()
  while 'errors' in vac:
        vac = requests.get(f'https://api.hh.ru/vacancies/{vacId}', headers=headersForVacancy).json()
  
  desc = vac['name'].lower() + ' '

  for k in vac['key_skills']:
      desc += k['name'].lower() + ' '
  # print(vac['key_skills'])
  soup = BeautifulSoup(vac['description'], 'html.parser')
  desc += soup.text.lower()
  token_desc1 = word_tokenize(desc, language='russian')
  # print(vac)
  # print(token_desc1)
  for token in token_desc1:
      if token not in stop_words and token not in punct and 'ə' not in token:
          stemmed_token = stemmer.stem(token)
          filtered_tokens.append(stemmed_token)
          all_filtered_tokens.add(stemmed_token) 
  # print(filtered_tokens)
  spec_words_vac = ' '.join(filtered_tokens)
  # print(spec_words_vac)
  spec_words_lst.append(spec_words_vac)
  lock2.release()

In [ ]:
def setVacancyInfo(vacancies, spec_words_lst, all_filtered_tokens):
  # using multithreading to get information from all vacancies
  
  lock2 = threading.Semaphore(100)
  thread_pool2 = []
  for vacId in set([vacancy['id'] for vacancy in vacancies]):
      thread = threading.Thread(target=getVacancy, args=(spec_words_lst, all_filtered_tokens, lock2, vacId))
      thread_pool2.append(thread)
      thread.start()
      lock2.acquire()

  for thread in thread_pool2:
      thread.join()
  # lock.release()



In [ ]:
headersForVacancy = {'Authorization': '#'}

lst = [i for i in range(113, 147)]

searchParams = {'period': '30', 'per_page': '100', 'responses_count_enabled': 'false'}

punct = string.punctuation.replace("#", "") + '—' + '”' + '“' + '``' + '«' + '»' + '•' + '/'

lock = threading.Semaphore(100)

def create_global_dict(specs_id_lst):
    # creating the global dictionary of most important words for all specializations
    
    global_spec_words_dict = {}
    for n in specs_id_lst:
        vacancies_ids = []
        
        spec_words_dict = {}
        spec_words_lst = []
        all_filtered_tokens = set()
        
        thread_pool = []
        t = time.time()
        for i in range(10):
            print(len(spec_words_lst))
            response = requests.get(f'https://api.hh.ru/vacancies/?period={searchParams["period"]}&per_page={searchParams["per_page"]}&responses_count_enabled={searchParams["responses_count_enabled"]}&page={i}&professional_role={n}', headers=headersForVacancy)
            setVacancyInfo(response.json()['items'], spec_words_lst, all_filtered_tokens)
        
        vectorizer = CountVectorizer(vocabulary=list(all_filtered_tokens))
        X = vectorizer.fit_transform(spec_words_lst)
        transformer = TfidfTransformer()
        X_tfidf = transformer.fit_transform(X)
        X_tfidf_mean = np.mean(X_tfidf.toarray(), axis=0)

        # print(len(spec_words_lst))
        spec_words_dict = dict(zip(all_filtered_tokens, X_tfidf_mean))
        sorted_spec_words_dict = {}
        
        
        sorted_keys = sorted(spec_words_dict, key=spec_words_dict.get, reverse=True)
        
        for w in sorted_keys:
            sorted_spec_words_dict[w] = spec_words_dict[w]

        # print(sorted_spec_words_dict)

        spec_words_dict_final = {}
        for index, key in enumerate(sorted_spec_words_dict):
            if index >= 500:
                break
            else:
                spec_words_dict_final[key] = sorted_spec_words_dict[key]
        # print(len(spec_words_dict_final))
        print(time.time() - t)
        global_spec_words_dict[n] = list(spec_words_dict_final.keys())
        print(len(global_spec_words_dict))
        print(global_spec_words_dict)
        try:
            with open("roles_dictionary.json", "a") as f:
                f.write(json.dumps(global_spec_words_dict, ensure_ascii=False))
        except Exception as e:
            print(e)
        finally:
            global_spec_words_dict.clear()
        


In [ ]:
# run only one time
# create_global_dict(lst)

In [ ]:
headersForResume = {'Authorization': '#'}

def getResume(id):
    # getting resume information
    
    resumeResponse = requests.get('https://api.hh.ru/resumes/' + id, headers=headersForResume)
    if resumeResponse.status_code == 200:
        return resumeResponse.json()
    else:
        return False

In [ ]:
resume = getResume('09c93be900004f2d620039ed1f736563726574')
role_id1 = [role['id'] for role in resume['professional_roles']]
print(role_id1)
print(resume)

['96']
{'last_name': None, 'first_name': None, 'middle_name': None, 'title': 'программист C#', 'created_at': '2008-12-16T09:36:17+0300', 'updated_at': '2021-12-06T16:26:44+0300', 'area': {'id': '4', 'name': 'Новосибирск', 'url': 'https://api.hh.ru/areas/4'}, 'age': 36, 'gender': {'id': 'male', 'name': 'Мужской'}, 'salary': {'amount': 300000, 'currency': 'RUR'}, 'photo': None, 'total_experience': {'months': 150}, 'certificate': [], 'hidden_fields': [], 'actions': {'download': {'pdf': {'url': 'https://api.hh.ru/resumes/09c93be900004f2d620039ed1f736563726574/download/%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20C%23.pdf?type=pdf'}, 'rtf': {'url': 'https://api.hh.ru/resumes/09c93be900004f2d620039ed1f736563726574/download/%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20C%23.rtf?type=rtf'}}}, 'alternate_url': 'https://hh.ru/resume/09c93be900004f2d620039ed1f736563726574', 'id': '09c93be900004f2d620039ed1f736563726574', 'download': {'pdf': {'url': '

In [ ]:
#09c93be900004f2d620039ed1f736563726574
def get_resume_vector(id):
    # text cleaning and vectorizing a resume
    
    filtered_tokens = []
    resume_words_lst = []
    
    resume = getResume(id)
    role_id1 = [role['id'] for role in resume['professional_roles']]
    string = resume['title'].lower() + ' '
    for word in resume['skill_set']:
        string += word.lower() + ' '
    for dct in resume['experience']:
        string += dct['description'].lower() + ' '
    if resume['skills']:
        string += resume['skills'].lower()
    token_str = word_tokenize(string, language='russian')
    for token in token_str:
          if token not in stop_words and token not in punct and 'ə' not in token:
              stemmed_token = stemmer.stem(token)
              filtered_tokens.append(stemmed_token)
    resume_words = ' '.join(filtered_tokens)
    with open("roles_dictionary.json", "r") as f:
        dictionary = json.loads(f.read())
    resume_words_lst.append(resume_words)
    vectorizer = CountVectorizer(vocabulary=dictionary[role_id1[0]])
    vector_resume = vectorizer.fit_transform(resume_words_lst)
    return vector_resume.toarray()

In [ ]:
get_resume_vector('09c93be900004f2d620039ed1f736563726574')

array([[ 0,  2,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  2,  7,  1,  0,
         0,  7,  1,  0, 10,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,
         0,  1,  0,  1,  0,  0,  2,  0,  8,  3,  0,  0,  0,  0,  0,  1,
         0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  6,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  1,  0,  0,  3,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  6,
         0,  0,  0,  2,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,
         0,  1,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  2,  1,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  9,  0,  0,  0,  0,  1,  0,  0,  0,
         0,  0,  0,  0,  3,  4,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [ ]:
def get_a_vacancy(id):
    # getting vacancy information
    
    vacancyResponse = requests.get('https://api.hh.ru/vacancies/' + id, headers=headersForVacancy)
    if vacancyResponse.status_code == 200:
        return vacancyResponse.json()
    else:
        return False

In [ ]:
get_a_vacancy('42222926')

{'id': '42222926',
 'premium': True,
 'billing_type': {'id': 'premium', 'name': 'Премиум'},
 'relations': [],
 'name': 'Разработчик C#',
 'insider_interview': None,
 'response_letter_required': False,
 'area': {'id': '3',
  'name': 'Екатеринбург',
  'url': 'https://api.hh.ru/areas/3'},
 'salary': {'from': 120000, 'to': None, 'currency': 'RUR', 'gross': False},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': {'city': 'Екатеринбург',
  'street': 'улица Шейнкмана',
  'building': '55',
  'description': None,
  'lat': 56.832034,
  'lng': 60.587539,
  'raw': 'Екатеринбург, улица Шейнкмана, 55',
  'metro': None,
  'metro_stations': []},
 'allow_messages': True,
 'site': {'id': 'hh', 'name': 'hh.ru'},
 'experience': {'id': 'between3And6', 'name': 'От 3 до 6 лет'},
 'schedule': {'id': 'remote', 'name': 'Удаленная работа'},
 'employment': {'id': 'full', 'name': 'Полная занятость'},
 'department': None,
 'contacts': {'name': 'Харченко Мария',
  'email': 'hr@knopka.com',
  'phones': [{'co

In [ ]:
def get_vacancy_vector(id):
    # text cleaning and vectorizing a vacancy
    
    filtered_tokens = []
    vacancy_words_lst = []
    
    vacancy = get_a_vacancy(id)
    role_id1 = [role['id'] for role in vacancy['professional_roles']]
    string = vacancy['name'].lower() + ' '
    for word in vacancy['key_skills']:
        string += word['name'].lower() + ' '
    soup = BeautifulSoup(vacancy['description'], 'html.parser')
    string += soup.text.lower()
    token_str = word_tokenize(string, language='russian')
    for token in token_str:
          if token not in stop_words and token not in punct and 'ə' not in token:
              stemmed_token = stemmer.stem(token)
              filtered_tokens.append(stemmed_token)
    vacancy_words = ' '.join(filtered_tokens)
    with open("roles_dictionary.json", "r") as f:
        dictionary = json.loads(f.read())
    vacancy_words_lst.append(vacancy_words)
    vectorizer = CountVectorizer(vocabulary=dictionary[role_id1[0]])
    vector_vacancy = vectorizer.fit_transform(vacancy_words_lst)
    return vector_vacancy.toarray()

In [ ]:
get_vacancy_vector('42222926')

array([[1, 9, 1, 2, 0, 5, 1, 5, 3, 5, 4, 1, 7, 2, 1, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 0, 4, 2, 1, 0, 7, 1, 1, 1, 2, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 6, 1, 2,
        0, 1, 3, 2, 0, 0, 0, 0, 1, 1, 0, 1, 2, 0, 0, 0, 2, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 1, 0, 0, 3, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 3, 2, 3, 1, 0,
        0, 2, 1, 0, 4, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 2, 0, 2, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 2, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 

In [ ]:
def get_cosine_similarity(vec1, vec2):
    # counting cosine similarity between two vectors
    
    similarity = 1 - spatial.distance.cosine(vec1, vec2)
    if similarity > 0.5:
        return 0.5
    return 1 - spatial.distance.cosine(vec1, vec2)

In [ ]:
get_cosine_similarity(get_vacancy_vector('42222926'), get_resume_vector('09c93be900004f2d620039ed1f736563726574'))

0.14222673479551629

In [ ]:
get_cosine_similarity(get_vacancy_vector('49869318'), get_resume_vector('09c93be900004f2d620039ed1f736563726574'))

0.14861295035835198

In [ ]:
get_cosine_similarity(get_vacancy_vector('49869318'), get_resume_vector('1a6443020008943f530039ed1f64483378476e'))

0.2845621368982486

In [ ]:
get_cosine_similarity(get_vacancy_vector('42222926'), get_resume_vector('1a6443020008943f530039ed1f64483378476e'))

0.23108538743337892

In [ ]:
get_cosine_similarity(get_vacancy_vector('48952810'), get_resume_vector('1a6443020008943f530039ed1f64483378476e'))

0.3019104835450963

In [ ]:
def get_key_skills_evaluation(res_id, vac_id):
    # getting a key skills similarity metric
    
    res = getResume(res_id)
    vac = get_a_vacancy(vac_id)
    res_lst = []
    vac_lst = []
    counter = 0
    
    for word in res['skill_set']:
        res_lst.append(word.lower())
    for word in vac['key_skills']:
        vac_lst.append(word['name'].lower())
    for i in res_lst:
        if i in vac_lst:
            counter += 1
    print(res_lst)
    print(vac_lst)
    if len(vac_lst) > 0:
        evaluation = (1/len(vac_lst)*counter)/2
    else:
        evaluation = 0
    if evaluation > 0.5:
        return 0.5
    return evaluation

In [ ]:
get_key_skills_evaluation('1a6443020008943f530039ed1f64483378476e', '48952810')

['ответственность', 'коммуникабельность', 'внимательность', 'трудолюбие', 'желание учиться', 'честность', 'грамотность', 'работа в команде', 'fiddler', 'api', 'test case', 'пользователь пк', 'html', 'css', 'qa', 'scrum', 'atlassian jira', 'тестирование', 'postman', 'json api', 'регресcионное тестирование', 'функциональное тестирование']
[]


0

In [ ]:
def get_evaluation(similarity, evaluation):
    # getting final evaluation of similarity between a resume and a vacancy
    
    return similarity + evaluation

In [ ]:
get_evaluation(get_cosine_similarity(get_vacancy_vector('42222926'), get_resume_vector('09c93be900004f2d620039ed1f736563726574')), get_key_skills_evaluation('09c93be900004f2d620039ed1f736563726574', '42222926'))

['ado.net', 'ajax', 'asn1', 'c#', 'certificate', 'delphi', 'dewexpress', 'entityframework', 'git', 'linq', 'ms silverlight', 'ms sql', 'my sql', 'net framework 2-4', 'oracle', 'php', 'spring.net', 'svn', 'telerik', 'tfs', 't-sql', 'vb', 'wcf', 'windows installer xml', 'wpf', 'xml', 'xpath', 'xsd', 'nhibernate', 'dapper']
['c#', '.net framework', 'sql', 'java']


0.2672267347955163